In [ ]:
import os # To get the path of this file
import scrapy # Library for the crawler
import json # To output our data in JSON format
import logging
import threading # For locks
import PyPDF2 # To parse PDF files when a HTML version is not available
import requests # Easier to get PDF files

# Scrapy specific imports
from scrapy.crawler import CrawlerProcess
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor

# Settings for notebook & scrapy
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Make a lock for counter value
lock = threading.RLock()

In [ ]:
SEARCH_KEYS = [ # The search terms we are using to retrieve information
    'cyber%20security',
#     'cybersecurity',
#     'cybercrime',
]

In [ ]:
class JsonWriterPipeline(object):
    """
    This pipeline will output all data in one JSON file, line separated.
    """

    def open_spider(self, spider):
        """
        When starting this pipeline, the file needs to be openend.
        """
        self.file = open('euresult.jl', 'w')

    def close_spider(self, spider):
        """
        When the crawler is done, the file needs to be closed.
        """
        self.file.close()

    def process_item(self, item, spider):
        """
        Every item that is processed is written to a new line in the file.
        """
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item
    

class JsonSeperateFileWriterPipeline(object):
    """
    This pipeline will output all data in seperate JSON files, using the ID as file name, storing in the directory 'files'.
    """
    
    def open_spider(self, spider):
        """
        We do not have to do anything when starting this pipeline
        """
        logging.info(msg="Starting JsonSeperateFileWriterPipeline")
        
    def close_spider(self, spider):
        """
        We print the statistics for this run when the spider quits
        """
        logging.info("Finished! We processed " + str(spider.item_counter) + " files.")
        logging.info("Closing JsonSeperateFileWriterPipeline")

    def process_item(self, item, spider):
        """
        Every item that is processed is written to a seperate json file.
        """
        # TODO: check if this approach works on Windows
        tmp = open(os.path.abspath(os.curdir) + '/files/{}.json'.format(item['local_id'].replace('/', '')), 'w')
        line = json.dumps(dict(item)) + "\n"
        
        tmp.write(line)
        tmp.close()
        return item

In [ ]:
class EucybsecSpider(CrawlSpider):
    """
    The spider designed for the European Union CELEX website.
    """
    
    name = 'eucybsec'
    allowed_domains = ['eur-lex.europa.eu'] # URLs outside these domains are not followed
    start_urls = [ # TODO: Soft-code search terms
        'https://eur-lex.europa.eu/search.html?text=cyber%20security&scope=EURLEX&type=quick&lang=en'
    ]
    
    custom_settings = {
        'LOG_LEVEL': logging.INFO,
        'ITEM_PIPELINES': {
#             '__main__.JsonWriterPipeline': 1,
            '__main__.JsonSeperateFileWriterPipeline': 1,
        }, 
    }
    
    rules = (
        Rule(
            LinkExtractor(
                allow=(),
                restrict_xpaths=('(//a[@title="Next Page"])[1]',), # This will extract the URL for the next page, if existent
            ),
            callback='parse_item',
            follow=True,
        ),
    )
    
    # Non scrapy variables
    item_counter = 0
    MAIN_DOMAIN = "https://eur-lex.europa.eu"

    def parse_document_page(self, response):
        """
        Yield the values for one specific document
        """
        
        # The neatest way to get the document date, but not always available
        document_date = response.xpath('//meta[@property="eli:date_document"]/@content').extract_first()
        
        # The other way to get the document date.
        if not document_date:
            document_date = response.xpath('//p[@class="hd-date"]/text()').extract_first()
            if document_date:
                document_date = document_date.strip()    
        
        # Compress the content for efficiency
        # TODO: See #5
        content_elements = response.xpath('//div[@class="tabContent"]//text()').extract()
        content = ' '.join(content_elements)
        
        # The HTML version is not available, so we are going to extract the PDF
        if content == "":
            
            content = self.pdf_retriever(response)
            
        
        # Minimize the content for efficiency
        content = ' '.join(content.split()).lower()
        
        with lock:
            self.item_counter += 1
            this_item = self.item_counter
        
        yield {
            'title': response.xpath("//p[@id='translatedTitle']/text()").extract_first(),
            'local_id': response.xpath('//*[contains(@class,"DocumentTitle")]/text()').extract_first().split()[1],
            'order_id': this_item,
            'document_type': response.xpath('//meta[@property="eli:type_document"]/@resource').extract_first(),
            'publication_date': response.xpath('//meta[@property="eli:date_publication"]/@content').extract_first(),
            'document_date': document_date,
            'direct_url': response.url,
            'content': content,
        }

    def parse_item(self, response):
        """
        Parsing a page with search results, calling for the next page in the end.
        """
        
#         print('Processing.. ' + response.url)
        item_links = response.xpath("//div[contains(@class,'SearchResult')]/h2/a/@href").extract()
            
        item_links = [self.MAIN_DOMAIN + rel_path[1:] for rel_path in item_links]

        for a in item_links:
            yield scrapy.Request(a, callback=self.parse_document_page)
            
    
    def parse_start_url(self, response):
        return self.parse_item(response)
    
    
    
    def pdf_retriever(self, response):
        # Get the relative URL of the PDF file
        pdf_href = response.xpath("//a[@id='format_language_table_PDF_EN']/@href").extract_first()
        
        if not pdf_href:
            logging.info("PDF URL not located.")
            return ""

        # Make the URL absolute
        pdf_href = self.MAIN_DOMAIN + pdf_href.split("..")[-1]
        
        logging.info(pdf_href)
        
        try:
            remoteFile = requests.get(pdf_href)
        except:
            logging.info("Could not get PDF.")
            return ""
        
        # Temporary store the PDF file locally
        with open("tmp.pdf", 'wb') as my_data:
            my_data.write(remoteFile.content)

        open_pdf_file = open("tmp.pdf", 'rb')
        
        try:
            pdfReader = PyPDF2.PdfFileReader(open_pdf_file)
        except Exception as e:
            logging.info("Could not read PDF. Error:")
            logging.info(e)
            return ""

        content = ""
        
        # append all the pages to the content
        for page_number in range(pdfReader.numPages):
            content += pdfReader.getPage(page_number).extractText()
        
        return content
            


In [ ]:
def run_crawler():
    """
    Start the crawler
    """
    process = CrawlerProcess({
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
    })

    process.crawl(EucybsecSpider)
    process.start()
    
# TODO: Figure out how to 'restart' without having to fully restart the kernel.
run_crawler()